# Rules Based Agents - SOLUTION



## Core
### Step 1: Setting Up the Environment

**Installs**: We will use `langgraph` library for our agents.

**Imports**: Use `TypedDict` for state typing, `StateGraph` and `END` for the workflow, and `random` for simulation.

**Why**: Keeps dependencies minimal, focusing on rule-based logic without external tools.



In [1]:
%pip install -qU langgraph==0.5.*

Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import random

### Step 2: Defining the Shared State

**Fields**:

`car_count`: Number of cars waiting (simulated).

`light_state`: Current light state (red, yellow, green).

`duration`: Time in seconds for the state.

`Purpose`: Tracks the intersection’s conditions and the agent’s decision.

`Why`: Simple state reflects Lab 5a’s structure, now for a single agent.



In [3]:

# Define the shared state
class State(TypedDict):
    car_count: int      # Number of cars waiting at the intersection
    light_state: str    # Current light state (red, yellow, green)
    duration: int       # Duration in seconds for the current state

### Step 3: Traffic Light Agent (Rule-Based Reasoning)

- **Function**: Applies deterministic rules to set the light state and duration.

- **Logic**:

    If `car_count > 5`: Green light for 30 seconds (high traffic).

    If `car_count < 2`: Red light for 20 seconds (low traffic).

    `Else (2-5 cars): Yellow light for 15 seconds (moderate traffic).

- **Simulation**: Randomly generates car_count (0-10) for demo purposes.

- **Output**: Updates state with the decision and prints the applied rule.

- **Why**: Demonstrates rule-based reasoning with clear, explicit conditions.



In [4]:


# Traffic Light Agent: Applies rule-based reasoning to control the light
def traffic_light_agent(state: State) -> dict:
    print("Traffic Light Agent: Evaluating traffic conditions...")
    # Simulate the number of cars waiting (random for demo purposes)
    state['car_count'] = random.randint(0, 10)
    cars = state['car_count']

    # Rule-based logic
    if cars > 5:
        state['light_state'] = "green"
        state['duration'] = 30
        print(f"Rule applied: More than 5 cars ({cars}) → Green for 30 seconds")
    elif cars < 2:
        state['light_state'] = "red"
        state['duration'] = 20
        print(f"Rule applied: Fewer than 2 cars ({cars}) → Red for 20 seconds")
    else:
        state['light_state'] = "yellow"
        state['duration'] = 15
        print(f"Rule applied: 2-5 cars ({cars}) → Yellow for 15 seconds")

    return state


### Step 4: Building the Workflow

**Graph**: Single node (traffic_light) with a direct path to END.

**Purpose**: Executes the agent once, keeping it simple for rule-based focus.

**Why**: Contrasts with multi-agent workflows from 5a/5b, emphasizing single-agent logic.



In [5]:


# Build the workflow graph (single agent)
workflow = StateGraph(State)
workflow.add_node("traffic_light", traffic_light_agent)
workflow.set_entry_point("traffic_light")
workflow.add_edge("traffic_light", END)

# Compile the app
app = workflow.compile()

### Step 5: Running the System

**Initialization**: Starts with empty state values.

**Execution**: Runs the workflow, simulating one traffic cycle.

**Output**: Displays the car count and light decision.

Now we'll test the app to check for a simulated case with a straightforward set of scenarios to illustrate the rules in action.



In [6]:


# Initialize state
initial_state = {
    "car_count": 0,
    "light_state": "",
    "duration": 0
}

for run in range(10):
    print(f"\nRun {run + 1}:")
    # Run the app
    result = app.invoke(initial_state)

    # Display results
    print("\nTraffic Light Decision:")
    print(f"Car count: {result['car_count']}")
    print(f"Light state: {result['light_state']} for {result['duration']} seconds")


Run 1:
Traffic Light Agent: Evaluating traffic conditions...
Rule applied: More than 5 cars (6) → Green for 30 seconds

Traffic Light Decision:
Car count: 6
Light state: green for 30 seconds

Run 2:
Traffic Light Agent: Evaluating traffic conditions...
Rule applied: More than 5 cars (9) → Green for 30 seconds

Traffic Light Decision:
Car count: 9
Light state: green for 30 seconds

Run 3:
Traffic Light Agent: Evaluating traffic conditions...
Rule applied: More than 5 cars (8) → Green for 30 seconds

Traffic Light Decision:
Car count: 8
Light state: green for 30 seconds

Run 4:
Traffic Light Agent: Evaluating traffic conditions...
Rule applied: Fewer than 2 cars (0) → Red for 20 seconds

Traffic Light Decision:
Car count: 0
Light state: red for 20 seconds

Run 5:
Traffic Light Agent: Evaluating traffic conditions...
Rule applied: 2-5 cars (5) → Yellow for 15 seconds

Traffic Light Decision:
Car count: 5
Light state: yellow for 15 seconds

Run 6:
Traffic Light Agent: Evaluating traffic c

### Step 6: Running Multiple Agents

You may be asking yourself,
> "Why do we need the sophistication of agents, when we could simply use a conditional (if-then-else) workflow to accomplish the same thing?"

However, the simplicity of a single agent with static rules doesn’t fully showcase why an agentic approach is valuable. Agents are most useful when the following elements are needed:

**Modularity**: Multiple components need to interact or be swapped out.

**State Management**: Complex or evolving state requires structured updates.

**Scalability**: The system might grow to include multiple agents or dynamic rules.

To justify agents here, let’s enhance the system by adding complexity that leverages LangGraph’s strengths, making it more than just control flow.



In [7]:
# Define the shared state
class State(TypedDict):
    ns_cars: int        # Cars waiting in North-South direction
    ew_cars: int        # Cars waiting in East-West direction
    ns_light: str       # North-South light state (red, green)
    ew_light: str       # East-West light state (red, green)
    duration: int       # Duration in seconds for the current state

# North-South Controller: Sets N-S light based on traffic and coordination
def ns_controller(state: State) -> dict:
    print("N-S Controller: Evaluating North-South traffic...")
    state['ns_cars'] = random.randint(0, 10)
    state['ew_cars'] = random.randint(0, 10)

    # Rule: Prioritize direction with more cars, but only one can be green
    if state['ns_cars'] > state['ew_cars'] and state['ew_light'] != "green":
        state['ns_light'] = "green"
        state['duration'] = 30
        print(f"Rule applied: N-S {state['ns_cars']} > E-W {state['ew_cars']} → N-S Green for 30s")
    else:
        state['ns_light'] = "red"
        state['duration'] = 20 if state['ew_light'] == "green" else 0
        print(f"Rule applied: N-S {state['ns_cars']} ≤ E-W {state['ew_cars']} or E-W green → N-S Red")
    return state

# East-West Controller: Sets E-W light based on traffic and N-S state
def ew_controller(state: State) -> dict:
    print("E-W Controller: Evaluating East-West traffic...")
    # Rule: Only set green if N-S isn’t green and E-W has priority or equal traffic
    if state['ns_light'] != "green" and (state['ew_cars'] >= state['ns_cars']):
        state['ew_light'] = "green"
        state['duration'] = 30
        print(f"Rule applied: E-W {state['ew_cars']} ≥ N-S {state['ns_cars']} and N-S not green → E-W Green for 30s")
    else:
        state['ew_light'] = "red"
        if state['duration'] == 0:  # Set duration if N-S didn’t
            state['duration'] = 20
        print(f"Rule applied: N-S green or E-W {state['ew_cars']} < N-S {state['ns_cars']} → E-W Red")
    return state

# Build the workflow graph
workflow = StateGraph(State)
workflow.add_node("ns_controller", ns_controller)
workflow.add_node("ew_controller", ew_controller)
workflow.set_entry_point("ns_controller")
workflow.add_edge("ns_controller", "ew_controller")
workflow.add_edge("ew_controller", END)

# Compile the app
app = workflow.compile()

# Initialize state
initial_state = {
    "ns_cars": 0,
    "ew_cars": 0,
    "ns_light": "red",
    "ew_light": "red",
    "duration": 0
}

for run in range(3):
    print(f"\nRun {run + 1}:")

    # Run the app
    result = app.invoke(initial_state)

    # Display results
    print("\nTraffic Light Decision:")
    print(f"North-South Cars: {result['ns_cars']}, Light: {result['ns_light']}")
    print(f"East-West Cars: {result['ew_cars']}, Light: {result['ew_light']}")
    print(f"Duration: {result['duration']} seconds")


Run 1:
N-S Controller: Evaluating North-South traffic...
Rule applied: N-S 10 > E-W 6 → N-S Green for 30s
E-W Controller: Evaluating East-West traffic...
Rule applied: N-S green or E-W 6 < N-S 10 → E-W Red

Traffic Light Decision:
North-South Cars: 10, Light: green
East-West Cars: 6, Light: red
Duration: 30 seconds

Run 2:
N-S Controller: Evaluating North-South traffic...
Rule applied: N-S 0 ≤ E-W 0 or E-W green → N-S Red
E-W Controller: Evaluating East-West traffic...
Rule applied: E-W 0 ≥ N-S 0 and N-S not green → E-W Green for 30s

Traffic Light Decision:
North-South Cars: 0, Light: red
East-West Cars: 0, Light: green
Duration: 30 seconds

Run 3:
N-S Controller: Evaluating North-South traffic...
Rule applied: N-S 9 > E-W 6 → N-S Green for 30s
E-W Controller: Evaluating East-West traffic...
Rule applied: N-S green or E-W 6 < N-S 9 → E-W Red

Traffic Light Decision:
North-South Cars: 9, Light: green
East-West Cars: 6, Light: red
Duration: 30 seconds


### Why Agents Make Sense Here

**Coordination**: Two agents share state (ns_light affects ew_light), requiring interaction that’s cleaner as separate nodes than nested conditionals.

**Modularity**: Each direction’s logic is encapsulated, making it easy to modify or extend (e.g., add a pedestrian light agent).

**Scalability**: LangGraph’s structure supports adding more agents or rules later.




## Challenge 1: Build a Thermostat Controller

**Context**: Design a system to control a smart thermostat based on temperature and occupancy using rule-based reasoning. Control thermostats for two zones (e.g. Living Room, Bedroom) with coordination.

**Goal**: Apply rule-based logic to a home automation scenario, reinforcing deterministic reasoning.

**Task**: Create a workflow with:

- **Thermostat Agent**: Applies rules like the following for both rooms

    - If `temp > 75`°F and `occupants > 0`: Set to “cool” at 70°F.

    - If `temp < 65`°F and `occupants > 0`: Set to “heat” at 72°F.

    - If `occupants = 0`: Set to “off.”

- **State**: `temp` (random 60-80°F), `occupants` (random 0-5), `mode`, `target_temp` needed for each room (you may want to define a state for each room)


- **Guidance**: Use a multi-agent graph, simulating temp and occupants for two rooms.


In [8]:
# YOUR CODE HERE
# [SOLUTION]

class State(TypedDict):
    lr_temp: int        # Living Room temperature
    br_temp: int        # Bedroom temperature
    occupants: int      # Total occupants (shared)
    lr_mode: str        # Living Room mode
    br_mode: str        # Bedroom mode
    lr_target: int      # Living Room target temp
    br_target: int      # Bedroom target temp

def lr_thermostat(state: State) -> dict:
    print("LR Thermostat: Evaluating Living Room...")
    state['lr_temp'] = random.randint(60, 80)
    state['br_temp'] = random.randint(60, 80)
    state['occupants'] = random.randint(0, 5)
    if state['lr_temp'] > 75 and state['occupants'] > 0 and state['br_mode'] != "cool":
        state['lr_mode'] = "cool"
        state['lr_target'] = 70
        print(f"Rule: LR {state['lr_temp']}°F > 75°F, {state['occupants']} occupants → Cool to 70°F")
    else:
        state['lr_mode'] = "off"
        state['lr_target'] = 0
        print(f"Rule: LR {state['lr_temp']}°F ≤ 75°F or no priority → Off")
    return state

def br_thermostat(state: State) -> dict:
    print("BR Thermostat: Evaluating Bedroom...")
    if state['br_temp'] > 75 and state['occupants'] > 0 and state['lr_mode'] != "cool":
        state['br_mode'] = "cool"
        state['br_target'] = 70
        print(f"Rule: BR {state['br_temp']}°F > 75°F, {state['occupants']} occupants → Cool to 70°F")
    elif state['br_temp'] < 65 and state['occupants'] > 0:
        state['br_mode'] = "heat"
        state['br_target'] = 72
        print(f"Rule: BR {state['br_temp']}°F < 65°F → Heat to 72°F")
    else:
        state['br_mode'] = "off"
        state['br_target'] = 0
        print(f"Rule: BR {state['br_temp']}°F in range or no occupants → Off")
    return state

workflow = StateGraph(State)
workflow.add_node("lr_thermostat", lr_thermostat)
workflow.add_node("br_thermostat", br_thermostat)
workflow.set_entry_point("lr_thermostat")
workflow.add_edge("lr_thermostat", "br_thermostat")
workflow.add_edge("br_thermostat", END)

app = workflow.compile()

initial_state = {
    "lr_temp": 0,
    "br_temp": 0,
    "occupants": 0,
    "lr_mode": "off",
    "br_mode": "off",
    "lr_target": 0,
    "br_target": 0
}

for i in range(3):
    print(f"\nRun {i + 1}:")
    # Run the app
    result = app.invoke(initial_state)

    print("\nThermostat Decision:")
    print(f"Living Room: Temp {result['lr_temp']}°F, Mode: {result['lr_mode']}, Target: {result['lr_target']}°F")
    print(f"Bedroom: Temp {result['br_temp']}°F, Mode: {result['br_mode']}, Target: {result['br_target']}°F")
    print(f"Occupants: {result['occupants']}")


Run 1:
LR Thermostat: Evaluating Living Room...
Rule: LR 68°F ≤ 75°F or no priority → Off
BR Thermostat: Evaluating Bedroom...
Rule: BR 76°F > 75°F, 3 occupants → Cool to 70°F

Thermostat Decision:
Living Room: Temp 68°F, Mode: off, Target: 0°F
Bedroom: Temp 76°F, Mode: cool, Target: 70°F
Occupants: 3

Run 2:
LR Thermostat: Evaluating Living Room...
Rule: LR 64°F ≤ 75°F or no priority → Off
BR Thermostat: Evaluating Bedroom...
Rule: BR 71°F in range or no occupants → Off

Thermostat Decision:
Living Room: Temp 64°F, Mode: off, Target: 0°F
Bedroom: Temp 71°F, Mode: off, Target: 0°F
Occupants: 1

Run 3:
LR Thermostat: Evaluating Living Room...
Rule: LR 65°F ≤ 75°F or no priority → Off
BR Thermostat: Evaluating Bedroom...
Rule: BR 75°F in range or no occupants → Off

Thermostat Decision:
Living Room: Temp 65°F, Mode: off, Target: 0°F
Bedroom: Temp 75°F, Mode: off, Target: 0°F
Occupants: 3



## Challenge 2: Build a Parking Lot Gate System (Entry/Exit)

**Context**: Design a system to control a parking lot gate based on car presence and space availability using rule-based reasoning and multi-agent coordination.

**Task**: Create a workflow with:

**Gate Agent**: Applies rules like:

- If `cars_waiting` > 0 and `spaces > 0`: Set gate to “open” for 10 seconds.

- If `spaces = 0`: Set gate to “closed.”

- If `cars_waiting` = 0: Set gate to “closed.”

**State**: `cars_waiting` (random 0-3), `spaces` (random 0-5), `gate`, `duration` for both **entry and exit gates**.

**Guidance**: Simulate inputs (randomized) and use a multi-agent workflow.



In [9]:
# YOUR CODE HERE
# [SOLUTION]

# Define the shared state
class State(TypedDict):
    cars_entering: int  # Cars waiting to enter
    cars_exiting: int   # Cars waiting to exit
    spaces: int         # Available spaces (max 5)
    entry_gate: str     # Entry gate state
    exit_gate: str      # Exit gate state
    duration: int       # Duration in seconds

# Entry Gate Agent: Controls the entry gate
def entry_gate_agent(state: State) -> dict:
    print("Entry Gate Agent: Evaluating entry conditions...")
    state['cars_entering'] = random.randint(0, 3)
    state['cars_exiting'] = random.randint(0, 2)
    state['spaces'] = random.randint(0, 5)
    if state['cars_entering'] > 0 and state['spaces'] > 0 and state['exit_gate'] != "open":
        state['entry_gate'] = "open"
        state['duration'] = 10
        print(f"Rule: {state['cars_entering']} entering, {state['spaces']} spaces → Entry Open for 10s")
    else:
        state['entry_gate'] = "closed"
        print(f"Rule: No cars or no spaces → Entry Closed")
    return state

# Exit Gate Agent: Controls the exit gate
def exit_gate_agent(state: State) -> dict:
    print("Exit Gate Agent: Evaluating exit conditions...")
    if state['cars_exiting'] > 0 and state['entry_gate'] != "open":
        state['exit_gate'] = "open"
        state['duration'] = 10 if state['duration'] == 0 else state['duration']
        print(f"Rule: {state['cars_exiting']} exiting, entry not open → Exit Open for 10s")
    else:
        state['exit_gate'] = "closed"
        if state['duration'] == 0:
            state['duration'] = 5
        print(f"Rule: No cars exiting or entry open → Exit Closed")
    return state

# Build the workflow graph
workflow = StateGraph(State)
workflow.add_node("entry_controller", entry_gate_agent)  # Renamed to avoid conflict
workflow.add_node("exit_controller", exit_gate_agent)    # Renamed to avoid conflict
workflow.set_entry_point("entry_controller")
workflow.add_edge("entry_controller", "exit_controller")
workflow.add_edge("exit_controller", END)

# Compile the app
app = workflow.compile()

# Initialize state
initial_state = {
    "cars_entering": 0,
    "cars_exiting": 0,
    "spaces": 0,
    "entry_gate": "closed",
    "exit_gate": "closed",
    "duration": 0
}

for run in range(3):
    print(f"\nRun {run + 1}:")
    # Run the app
    result = app.invoke(initial_state)

    # Display results
    print("\nParking Lot Gate Decision:")
    print(f"Cars Entering: {result['cars_entering']}, Entry Gate: {result['entry_gate']}")
    print(f"Cars Exiting: {result['cars_exiting']}, Exit Gate: {result['exit_gate']}")
    print(f"Available Spaces: {result['spaces']}")
    print(f"Duration: {result['duration']} seconds")


Run 1:
Entry Gate Agent: Evaluating entry conditions...
Rule: No cars or no spaces → Entry Closed
Exit Gate Agent: Evaluating exit conditions...
Rule: 2 exiting, entry not open → Exit Open for 10s

Parking Lot Gate Decision:
Cars Entering: 0, Entry Gate: closed
Cars Exiting: 2, Exit Gate: open
Available Spaces: 2
Duration: 10 seconds

Run 2:
Entry Gate Agent: Evaluating entry conditions...
Rule: 2 entering, 4 spaces → Entry Open for 10s
Exit Gate Agent: Evaluating exit conditions...
Rule: No cars exiting or entry open → Exit Closed

Parking Lot Gate Decision:
Cars Entering: 2, Entry Gate: open
Cars Exiting: 1, Exit Gate: closed
Available Spaces: 4
Duration: 10 seconds

Run 3:
Entry Gate Agent: Evaluating entry conditions...
Rule: No cars or no spaces → Entry Closed
Exit Gate Agent: Evaluating exit conditions...
Rule: No cars exiting or entry open → Exit Closed

Parking Lot Gate Decision:
Cars Entering: 3, Entry Gate: closed
Cars Exiting: 0, Exit Gate: closed
Available Spaces: 0
Durati